In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import warnings
import requests
import time
warnings.filterwarnings('ignore')

class SNP500StockScreener:
    def __init__(self):
        self.sp500_tickers = []
        self.stock_data = {}
        self.factor_scores = pd.DataFrame()
        self.final_rankings = pd.DataFrame()

    def get_sp500_tickers(self):
        """
        Fetch S&P 500 tickers with multiple fallback methods
        """
        print("Fetching S&P 500 tickers")

        # Method 1: Wikipedia Scraping
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

            response = requests.get(url, headers=headers, timeout=10)
            tables = pd.read_html(response.text)
            sp500_table = tables[0]

            tickers = sp500_table['Symbol'].tolist()
            tickers = [ticker.replace('.', '-') for ticker in tickers]

            self.sp500_tickers = tickers
            print(f"Successfully loaded {len(tickers)} from Wikipedia")
            return tickers

        except Exception as e:
            print(f"Wikipedia method failed: {e}")

        # Method 2: yfinance
        try:
            print("Trying alternative method: Fetching from SPY ETF...")
            spy = yf.Ticker("SPY")
            holdings = spy.get_holdings()

            if holdings is not None and len(holdings) > 0:
                tickers = holdings.index.tolist()[:500]  # Top 500
                self.sp500_tickers = tickers
                print(f"Successfully loaded {len(tickers)} from yfinance")
                return tickers
        except Exception as e:
            print(f"yfinance method failed: {e}")

        # Method 3: manual list
        print("using fallback manual list")
        self.sp500_tickers = self._get_manual_sp500_list()
        print(f"Loaded {len(self.sp500_tickers)} tickers from manual list")
        return self.sp500_tickers

    def _get_manual_sp500_list(self): #using manual list
        sp500_tickers = [
            # Mega Cap Tech
            'AAPL', 'MSFT', 'GOOGL', 'GOOG', 'AMZN', 'NVDA', 'META', 'TSLA',
            'BRK-B', 'AVGO', 'LLY', 'JPM', 'V', 'UNH', 'XOM', 'MA', 'COST',
            'HD', 'PG', 'JNJ', 'NFLX', 'ABBV', 'BAC', 'CRM', 'ORCL', 'CVX',
            'KO', 'AMD', 'MRK', 'PEP', 'ADBE', 'TMO', 'WMT', 'CSCO', 'ACN',
            'MCD', 'ABT', 'NKE', 'LIN', 'DHR', 'QCOM', 'TXN', 'PM', 'INTC',
            'VZ', 'DIS', 'INTU', 'CMCSA', 'AMGN', 'IBM', 'UNP', 'NEE', 'RTX',
            'HON', 'COP', 'SPGI', 'GE', 'LOW', 'CAT', 'UBER', 'AXP', 'BA',
            'T', 'PLD', 'DE', 'BLK', 'GS', 'NOW', 'MS', 'ELV', 'AMAT', 'BKNG',
            'SYK', 'PFE', 'VRTX', 'TJX', 'ADP', 'SBUX', 'GILD', 'MMC', 'ADI',
            'MDLZ', 'LRCX', 'C', 'AMT', 'REGN', 'CVS', 'CI', 'ISRG', 'SCHW',

            # Large Cap
            'ZTS', 'PGR', 'MO', 'CB', 'SO', 'BDX', 'DUK', 'BMY', 'PANW', 'NOC',
            'ETN', 'APH', 'BSX', 'CME', 'TT', 'ITW', 'KLAC', 'MCO', 'SNPS',
            'WM', 'EOG', 'ICE', 'HCA', 'USB', 'CL', 'MSI', 'FI', 'SHW', 'EMR',
            'CSX', 'GD', 'MCK', 'NSC', 'APD', 'PSA', 'MAR', 'PH', 'AON', 'ECL',
            'PYPL', 'MMM', 'ORLY', 'TDG', 'BK', 'AJG', 'TGT', 'PCAR', 'GM',
            'JCI', 'ROP', 'AFL', 'MET', 'SLB', 'HUM', 'ADSK', 'SRE', 'CARR',
            'NXPI', 'TRV', 'AIG', 'ALL', 'MNST', 'AEP', 'CMG', 'AZO', 'ROST',
            'MCHP', 'KMB', 'O', 'D', 'EW', 'PAYX', 'PRU', 'KMI', 'SPG', 'EXC',
            'MSCI', 'WELL', 'CCI', 'TEL', 'FAST', 'HES', 'CPRT', 'GWW', 'ODFL',
            'CTVA', 'F', 'CTAS', 'A', 'KDP', 'LHX', 'EA', 'DXCM', 'YUM', 'DLR',

            # Mid Cap
            'PCG', 'IQV', 'DHI', 'VRSK', 'XEL', 'RSG', 'AME', 'FANG', 'ED',
            'DD', 'ACGL', 'CHTR', 'WMB', 'MTD', 'KHC', 'IDXX', 'DAL', 'EXR',
            'VICI', 'ROK', 'ANSS', 'RMD', 'GLW', 'IR', 'OTIS', 'WEC', 'ON',
            'CDNS', 'CSGP', 'EBAY', 'GEHC', 'DOW', 'KEYS', 'CDW', 'IT', 'GPN',
            'EIX', 'FICO', 'STZ', 'VMC', 'AWK', 'MLM', 'PPG', 'MPWR', 'NUE',
            'CHD', 'RJF', 'FITB', 'WBD', 'WY', 'BKR', 'DFS', 'NDAQ', 'HPQ',
            'IFF', 'ETR', 'SBAC', 'ZBH', 'APTV', 'NTRS', 'WDC', 'PTC', 'CAH',
            'MTB', 'FDS', 'LYB', 'TSCO', 'LUV', 'TTWO', 'INVH', 'TROW', 'LH',
            'FTV', 'STT', 'BAX', 'BR', 'HAL', 'HPE', 'ARE', 'STLD', 'ALGN',
            'EPAM', 'SYF', 'TYL', 'CBRE', 'K', 'EFX', 'RF', 'NTAP', 'MKC',

            # Additional
            'ESS', 'DTE', 'MAA', 'IRM', 'EQR', 'PKG', 'BALL', 'HBAN', 'SWK',
            'EXPE', 'AEE', 'DOV', 'VTR', 'EQT', 'IP', 'PPL', 'HOLX', 'CFG',
            'WAB', 'OMC', 'AVB', 'CNP', 'DGX', 'CINF', 'LEN', 'CLX', 'TDY',
            'FE', 'VLTO', 'CBOE', 'ATO', 'MAS', 'ZBRA', 'WAT', 'TER', 'JBHT',
            'PFG', 'NVR', 'J', 'DRI', 'UAL', 'COF', 'CAG', 'LDOS', 'LVS',
            'TRMB', 'DPZ', 'BLDR', 'STE', 'HIG', 'HSY', 'XYL', 'TSN', 'BBY',
            'GPC', 'POOL', 'CE', 'LKQ', 'TXT', 'AKAM', 'CMS', 'PKI', 'SNA',
            'CPT', 'MOS', 'JKHY', 'LNT', 'EVRG', 'SWKS', 'GL', 'NDSN', 'FFIV',
            'HSIC', 'NI', 'GNRC', 'UDR', 'TAP', 'CPB', 'CHRW', 'BXP', 'MTCH',
            'BG', 'AIZ', 'HWM', 'AAL', 'PARA', 'FOXA', 'FOX', 'ALB', 'HII',
            'TPR', 'RL', 'AOS', 'WYNN', 'IVZ', 'NWSA', 'NWS', 'ZION', 'RHI',
            'PNW', 'WHR', 'BWA', 'NCLH', 'HAS', 'REG', 'KIM', 'FRT', 'VNO',
        ]

        return sp500_tickers

    def download_stock_data(self, lookback_months=36, batch_size=50, delay=1):
        """
        Download historical data with rate limiting and batch processing

        batch_size: Number of stocks to download before pausing
        delay: Seconds to wait between batches
        """
        print(f"\nDownloading stock data for {len(self.sp500_tickers)} tickers...")
        print("Using batch processing to avoid rate limits...")

        end_date = datetime.now()
        start_date = end_date - timedelta(days=lookback_months*30)

        successful_downloads = 0
        failed_tickers = []

        # Process in batches
        total_tickers = len(self.sp500_tickers)

        for batch_start in range(0, total_tickers, batch_size):
            batch_end = min(batch_start + batch_size, total_tickers)
            batch_tickers = self.sp500_tickers[batch_start:batch_end]

            print(f"\nProcessing batch {batch_start//batch_size + 1}: Tickers {batch_start+1}-{batch_end}")

            for i, ticker in enumerate(batch_tickers):
                try:
                    # Progress indicator
                    overall_progress = batch_start + i + 1
                    if overall_progress % 25 == 0:
                        print(f"Overall progress: {overall_progress}/{total_tickers} tickers ({successful_downloads} successful)")

                    # Download with timeout
                    stock = yf.Ticker(ticker)

                    # Download price history
                    hist = stock.history(start=start_date, end=end_date, timeout=10)

                    if len(hist) < 252:  # Need at least 1 year of data
                        failed_tickers.append(ticker)
                        continue

                    # Get fundamental data
                    info = stock.info

                    # Validate we have minimum required data
                    if not info or 'marketCap' not in info:
                        failed_tickers.append(ticker)
                        continue

                    # Store data
                    self.stock_data[ticker] = {
                        'history': hist,
                        'info': info
                    }

                    successful_downloads += 1

                    # delay to avoid overwhelming the API
                    time.sleep(0.1)

                except Exception as e:
                    failed_tickers.append(ticker)
                    continue

            # Pause between batches to avoid rate limiting
            if batch_end < total_tickers:
                print(f"Pausing {delay} seconds before next batch...")
                time.sleep(delay)

        print(f"\n{'='*60}")
        print(f"DOWNLOAD COMPLETE")
        print(f"{'='*60}")
        print(f"Successfully downloaded: {successful_downloads} stocks")
        print(f"Failed downloads: {len(failed_tickers)} stocks")

        if len(failed_tickers) > 0 and len(failed_tickers) < 20:
            print(f"Failed tickers: {', '.join(failed_tickers)}")

        return successful_downloads

    def download_stock_data_parallel(self, lookback_months=36, max_workers=5):
        from concurrent.futures import ThreadPoolExecutor, as_completed

        print(f"\nDownloading stock data using {max_workers} parallel workers...")

        end_date = datetime.now()
        start_date = end_date - timedelta(days=lookback_months*30)

        successful_downloads = 0
        failed_tickers = []

        def download_single_stock(ticker):
            """Helper function to download single stock"""
            try:
                stock = yf.Ticker(ticker)
                hist = stock.history(start=start_date, end=end_date, timeout=10)

                if len(hist) < 252:
                    return ticker, None, "Insufficient data"

                info = stock.info

                if not info or 'marketCap' not in info:
                    return ticker, None, "No fundamental data"

                return ticker, {'history': hist, 'info': info}, None

            except Exception as e:
                return ticker, None, str(e)

        # Download in parallel
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_ticker = {
                executor.submit(download_single_stock, ticker): ticker
                for ticker in self.sp500_tickers
            }

            for i, future in enumerate(as_completed(future_to_ticker)):
                ticker, data, error = future.result()

                if data is not None:
                    self.stock_data[ticker] = data
                    successful_downloads += 1
                else:
                    failed_tickers.append(ticker)

                # Progress update
                if (i + 1) % 50 == 0:
                    print(f"Progress: {i+1}/{len(self.sp500_tickers)} ({successful_downloads} successful)")

        print(f"\nSuccessfully downloaded: {successful_downloads} stocks")
        print(f"Failed downloads: {len(failed_tickers)} stocks")

        return successful_downloads


    def calculate_momentum_score(self, ticker):
        """Calculate 12-month momentum excluding most recent month"""
        try:
            hist = self.stock_data[ticker]['history']

            if len(hist) < 252:
                return np.nan

            price_12m_ago = hist['Close'].iloc[-252]
            price_1m_ago = hist['Close'].iloc[-21]

            momentum_return = (price_1m_ago - price_12m_ago) / price_12m_ago

            returns = hist['Close'].pct_change().dropna()
            volatility = returns.std() * np.sqrt(252)

            if volatility > 0:
                risk_adjusted_momentum = momentum_return / volatility
            else:
                risk_adjusted_momentum = 0

            return risk_adjusted_momentum

        except Exception as e:
            return np.nan

    def calculate_quality_score(self, ticker):
        """Calculate quality score"""
        try:
            info = self.stock_data[ticker]['info']

            roe = info.get('returnOnEquity', 0) * 100 if info.get('returnOnEquity') else 0

            market_cap = info.get('marketCap', 1)
            free_cash_flow = info.get('freeCashflow', 0)
            fcf_yield = (free_cash_flow / market_cap * 100) if market_cap > 0 else 0

            debt_to_equity = info.get('debtToEquity', 100) / 100 if info.get('debtToEquity') else 1

            operating_margin = info.get('operatingMargins', 0) * 100 if info.get('operatingMargins') else 0

            roe_score = min(roe / 15 * 25, 25)
            fcf_score = min(fcf_yield / 5 * 25, 25)
            debt_score = max(25 - debt_to_equity * 10, 0)
            margin_score = min(operating_margin / 20 * 25, 25)

            quality_score = roe_score + fcf_score + debt_score + margin_score

            return quality_score

        except Exception as e:
            return np.nan

    def calculate_value_score(self, ticker):
        """Calculate value score"""
        try:
            info = self.stock_data[ticker]['info']

            market_cap = info.get('marketCap', 1)
            free_cash_flow = info.get('freeCashflow', 0)
            fcf_yield = (free_cash_flow / market_cap * 100) if market_cap > 0 else 0

            price_to_book = info.get('priceToBook', 10)
            book_yield = (1 / price_to_book * 100) if price_to_book > 0 else 0

            pe_ratio = info.get('trailingPE', 30)
            earnings_yield = (1 / pe_ratio * 100) if pe_ratio > 0 else 0

            fcf_score = min(fcf_yield / 8 * 40, 40)
            book_score = min(book_yield / 15 * 30, 30)
            earnings_score = min(earnings_yield / 5 * 30, 30)

            value_score = fcf_score + book_score + earnings_score

            return value_score

        except Exception as e:
            return np.nan

    def calculate_low_volatility_score(self, ticker):
        """Calculate low volatility score"""
        try:
            hist = self.stock_data[ticker]['history']

            returns = hist['Close'].pct_change().dropna()
            volatility = returns.std() * np.sqrt(252)

            beta = self.stock_data[ticker]['info'].get('beta', 1.0)

            vol_score = max(100 - (volatility * 200), 0)
            beta_score = max(100 - (beta * 50), 0)

            low_vol_score = (vol_score * 0.7 + beta_score * 0.3)

            return low_vol_score

        except Exception as e:
            return np.nan

    def calculate_liquidity_score(self, ticker):
        """Calculate liquidity score"""
        try:
            hist = self.stock_data[ticker]['history']

            avg_volume = hist['Volume'].iloc[-60:].mean()
            current_price = hist['Close'].iloc[-1]
            dollar_volume = avg_volume * current_price

            liquidity_score = min(dollar_volume / 10_000_000 * 100, 100)

            return liquidity_score

        except Exception as e:
            return np.nan

    def screen_stocks(self, factor_weights=None):
      """Main screening function with more flexible criteria"""
      if factor_weights is None:
          factor_weights = {
              'momentum': 0.40,
              'quality': 0.30,
              'value': 0.20,
              'low_vol': 0.10
          }

      print("\nCalculating factor scores for all stocks...")

      results = []

      for i, ticker in enumerate(self.stock_data.keys()):
          if (i + 1) % 50 == 0:
              print(f"Scoring progress: {i+1}/{len(self.stock_data)} stocks")

          try:
              momentum = self.calculate_momentum_score(ticker)
              quality = self.calculate_quality_score(ticker)
              value = self.calculate_value_score(ticker)
              low_vol = self.calculate_low_volatility_score(ticker)
              liquidity = self.calculate_liquidity_score(ticker)

              info = self.stock_data[ticker]['info']
              current_price = self.stock_data[ticker]['history']['Close'].iloc[-1]
              company_name = info.get('longName', ticker)
              sector = info.get('sector', 'Unknown')

              results.append({
                  'Ticker': ticker,
                  'Company': company_name,
                  'Sector': sector,
                  'Price': round(current_price, 2),
                  'Momentum_Score': momentum,
                  'Quality_Score': quality,
                  'Value_Score': value,
                  'LowVol_Score': low_vol,
                  'Liquidity_Score': liquidity
              })

          except Exception as e:
              continue

      self.factor_scores = pd.DataFrame(results)

      print(f"Total stocks before filtering: {len(self.factor_scores)}")

      # Count how many valid scores each stock has
      score_columns = ['Momentum_Score', 'Quality_Score', 'Value_Score', 'LowVol_Score']
      self.factor_scores['Valid_Scores'] = self.factor_scores[score_columns].notna().sum(axis=1)

      # Keep stocks with at least 3 out of 4 valid scores
      self.factor_scores = self.factor_scores[self.factor_scores['Valid_Scores'] >= 3]

      print(f"Stocks after filtering (3+ valid scores): {len(self.factor_scores)}")

      # Fill missing scores with median
      for factor in score_columns:
          median_score = self.factor_scores[factor].median()
          self.factor_scores[factor].fillna(median_score, inplace=True)

      print("\nNormalizing factor scores...")

      # Normalize to z-scores
      for factor in score_columns:
          mean = self.factor_scores[factor].mean()
          std = self.factor_scores[factor].std()
          if std > 0:
              self.factor_scores[f'{factor}_Z'] = (self.factor_scores[factor] - mean) / std
          else:
              self.factor_scores[f'{factor}_Z'] = 0

      # Calculate composite score
      self.factor_scores['Composite_Score'] = (
          self.factor_scores['Momentum_Score_Z'] * factor_weights['momentum'] +
          self.factor_scores['Quality_Score_Z'] * factor_weights['quality'] +
          self.factor_scores['Value_Score_Z'] * factor_weights['value'] +
          self.factor_scores['LowVol_Score_Z'] * factor_weights['low_vol']
      )

      # filter
      if len(self.factor_scores) > 10:
          liquidity_threshold = self.factor_scores['Liquidity_Score'].quantile(0.20)
          before_liq_filter = len(self.factor_scores)
          self.factor_scores = self.factor_scores[
              self.factor_scores['Liquidity_Score'] >= liquidity_threshold
          ]
          print(f"After liquidity filter (removed bottom 20%): {len(self.factor_scores)} stocks")

      # rank final
      self.factor_scores['Rank'] = self.factor_scores['Composite_Score'].rank(ascending=False)

      # sort by comp score
      self.final_rankings = self.factor_scores.sort_values('Composite_Score', ascending=False)

      print(f"final rankings: {len(self.final_rankings)} stocks")

      return self.final_rankings

    def get_top_stocks(self, n=50, min_composite_percentile=None):
        """
        Get top N stocksE

        n: Number of stocks to return
        min_composite_percentile: Optional filter (e.g., 0.5 for top 50%)
        """
        df = self.final_rankings.copy()

        # Optional to Filter by composite score percentile only
        if min_composite_percentile:
            threshold = df['Composite_Score'].quantile(1 - min_composite_percentile)
            df = df[df['Composite_Score'] >= threshold]
            print(f"After percentile filter: {len(df)} stocks remain")

        # return top N by composite score
        top_stocks = df.head(n)

        return top_stocks

    def get_stock_pick_summary(self, top_n=50):
        """
        Generate summary report
        """
        # REMOVED the strict min_per_factor requirement
        top_stocks = self.get_top_stocks(n=top_n)

        print(f"\n{'='*80}")
        print(f"TOP {top_n} Stock Picks")
        print(f"{'='*80}\n")

        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        pd.set_option('display.max_colwidth', 30)

        display_df = top_stocks[[
            'Rank', 'Ticker', 'Company', 'Sector', 'Price',
            'Momentum_Score', 'Quality_Score', 'Value_Score',
            'LowVol_Score', 'Composite_Score'
        ]].copy()

        for col in ['Momentum_Score', 'Quality_Score', 'Value_Score', 'LowVol_Score', 'Composite_Score']:
            display_df[col] = display_df[col].round(2)

        print(display_df.to_string(index=False))

        print(f"\n{'='*80}")
        print("sector distribution")
        print(f"{'='*80}\n")
        sector_dist = top_stocks['Sector'].value_counts()
        print(sector_dist)

        print(f"\n{'='*80}")
        print("factor score analysis")
        print(f"{'='*80}\n")
        factor_stats = top_stocks[[
            'Momentum_Score', 'Quality_Score', 'Value_Score', 'LowVol_Score'
        ]].describe()
        print(factor_stats.round(2))

        # Show high-momentum stocks
        print(f"\n{'='*80}")
        print("top 10 momentum stocks")
        print(f"{'='*80}\n")
        momentum_leaders = top_stocks.nlargest(10, 'Momentum_Score')[
            ['Ticker', 'Company', 'Momentum_Score', 'Composite_Score']
        ]
        print(momentum_leaders.to_string(index=False))

        return top_stocks

    def analyze_specific_stocks(self, tickers_to_check):
        """
        Check specific stocks if its on the list
        """
        print(f"\n{'='*80}")
        print(f"Analyzing sspecific stockpick")
        print(f"{'='*80}\n")

        if len(self.final_rankings) == 0:
            print("No rankings available. Run screen_stocks() first.")
            return

        for ticker in tickers_to_check:
            if ticker in self.final_rankings['Ticker'].values:
                stock_data = self.final_rankings[self.final_rankings['Ticker'] == ticker].iloc[0]

                print(f"\n{ticker} - {stock_data['Company']}")
                print(f"{'-'*60}")
                print(f"Rank: #{int(stock_data['Rank'])} out of {len(self.final_rankings)}")
                print(f"Composite Score: {stock_data['Composite_Score']:.2f}")
                print(f"\nFactor Breakdown:")
                print(f"  Momentum:  {stock_data['Momentum_Score']:.2f} (Z-score: {stock_data['Momentum_Score_Z']:.2f})")
                print(f"  Quality:   {stock_data['Quality_Score']:.2f} (Z-score: {stock_data['Quality_Score_Z']:.2f})")
                print(f"  Value:     {stock_data['Value_Score']:.2f} (Z-score: {stock_data['Value_Score_Z']:.2f})")
                print(f"  Low Vol:   {stock_data['LowVol_Score']:.2f} (Z-score: {stock_data['LowVol_Score_Z']:.2f})")
                print(f"  Liquidity: {stock_data['Liquidity_Score']:.2f}")
            else:
                print(f"\n{ticker}: Not found in rankings (may have been filtered out)")

    def export_results(self, filename='stock_picks.csv'):
        """
        Export the final rankings to a CSV file.
        """
        if not self.final_rankings.empty:
            self.final_rankings.to_csv(filename, index=False)
            print(f"\n Results successfully exported to {filename}")
        else:
            print("\n No rankings to export. Run screen_stocks() first.")

#main func to run

if __name__ == "__main__":

    print("="*80)
    print("S&P 500 MULTI-FACTOR STOCK SCREENING ALGORITHM")
    print("="*80)

    screener = SNP500StockScreener()

    # get tickers
    tickers = screener.get_sp500_tickers()

    # download stock data
    success_count = screener.download_stock_data(
        lookback_months=36,
        batch_size=50,
        delay=1
    )

    if success_count < 50:
        print("\n⚠ WARNING: Very few stocks downloaded.")
        response = input("\nContinue with available data? (y/n): ")
        if response.lower() != 'y':
            exit()

    # criteria
    factor_weights = {
        'momentum': 0.40,
        'quality': 0.30,
        'value': 0.20,
        'low_vol': 0.10
    }

    rankings = screener.screen_stocks(factor_weights=factor_weights)

    # get 50 picks
    top_picks = screener.get_top_stocks()

    # export
    screener.export_results('sp500_stock_picks.csv')

    print("\n" + "="*80)
    print("SCREENING COMPLETE!")
    print("="*80)
    print(f"\nTotal stocks ranked: {len(rankings)}")
    print(f"Top picks selected: {len(top_picks)}")
    print(f"Results saved")

S&P 500 MULTI-FACTOR STOCK SCREENING ALGORITHM
Fetching S&P 500 tickers...
✓ Successfully loaded 501 S&P 500 tickers from Wikipedia

Using batch processing to avoid rate limits...

Processing batch 1: Tickers 1-50
Overall progress: 25/501 tickers (24 successful)
Overall progress: 50/501 tickers (49 successful)
Pausing 1 seconds before next batch...

Processing batch 2: Tickers 51-100
Overall progress: 75/501 tickers (74 successful)
Overall progress: 100/501 tickers (99 successful)
Pausing 1 seconds before next batch...

Processing batch 3: Tickers 101-150
Overall progress: 125/501 tickers (124 successful)
Overall progress: 150/501 tickers (149 successful)
Pausing 1 seconds before next batch...

Processing batch 4: Tickers 151-200
Overall progress: 175/501 tickers (174 successful)
Overall progress: 200/501 tickers (199 successful)
Pausing 1 seconds before next batch...

Processing batch 5: Tickers 201-250
Overall progress: 225/501 tickers (224 successful)
Overall progress: 250/501 ticke